In [ ]:
# CROPPING#
#
# Variables for paths
input_dir = "/home/sastocke/nnUNet/inferenceTest/DataSet1Test"
output_binary_masks = "/home/sastocke/nnUNet/inferenceTest/DataSet2OutputFromnnUnet1"

import os

# Set the environment variables
os.environ["nnUNet_raw"] = "/home/sastocke/nnUNet/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/home/sastocke/nnUNet/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/home/sastocke/nnUNet/nnUNet_results"


#!nnUNetv2_predict -i /home/sastocke/nnUNet/inferenceTest/DataSet1Test -o /home/sastocke/nnUNet/inferenceTest/DataSet2OutputFromnnUnet1 -d 25 -c '2d'
os.system((f"nnUNetv2_predict -i {input_dir} -o {output_binary_masks}  --save_probabilities -d 25 -c '2d' -f 5"))

#!nnUNetv2_predict -i {input_dir} -o
#{output_binary_masks}  --save_probabilities -d 25 -c '2d' 



#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting Hannum_Volunteer_01_DiVO_06_10_slice_001:
perform_everything_on_device: True


100%|██████████| 1/1 [00:08<00:00,  8.89s/it]


sending off prediction to background worker for resampling and export
done with Hannum_Volunteer_01_DiVO_06_10_slice_001


0

In [ ]:
#Proccessing Cropping:
import os
import nibabel as nib
import numpy as np
import cv2
import pandas as pd

# Define directories
#root directory
pwd = '/home/sastocke/nnUNet/inferenceTest'
#annotators
root_folders = ['Hannum']
datasetname = 'processedOutPut1'
output_image_folder = f'{pwd}/{datasetname}/imagesTr'
inspection_folder = f'{pwd}/{datasetname}/inspection'


inference_masks_dir = "/home/sastocke/nnUNet/inferenceTest/DataSet2OutputFromnnUnet1"  # First nnUNet mask outputs

# Ensure output folders exist
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(inspection_folder, exist_ok=True)

# Interpolation size
interpolation_size = 256

# Function to normalize image to [0, 1]
def normalize_image(image):
    min_val = np.min(image)
    max_val = np.max(image)
    return (image - min_val) / (max_val - min_val) if max_val > min_val else image

# Processing loop

#mask_file = os.path.join(inference_masks_dir, f'{root_folder}_{volunteer_folder}_{divo_folder}_slice_00{i}.nii.gz')

for mask_filename in os.listdir(inference_masks_dir):
    if mask_filename.endswith('.nii.gz'):  # Only process .nii.gz files
        # Extract relevant components from the filename
        try:
                # Extract components using the known file pattern
                parts = mask_filename.rstrip('.nii.gz').split('_')
                root_folder = parts[0]  # 'Hannum'
                volunteer_folder = f"{parts[1]}_{parts[2]}"  # 'Volunteer_01'
                divo_folder = f"{parts[3]}_{parts[4]}_{parts[5]}"  # 'DiVO_06_10'
                slice = f"{parts[7]}"  # '001'



                # Log the components for debugging
                print(f'parts: {parts}')
                print(f"Processing mask file: {mask_filename}")
                print(f"Root folder: {root_folder}, Volunteer: {volunteer_folder}, DIVO folder: {divo_folder}, Slice: {slice}")
            
            # Construct full path to the mask file
                mask_file_path = os.path.join(inference_masks_dir, mask_filename)
            
            # Continue with your nested loops and processing here
        except ValueError:
            print(f"Filename format does not match expected pattern: {mask_filename}")


        #Path found: 
        print(f'Path of mask: {root_folder}_{volunteer_folder}_{divo_folder}_{slice}')
        divo_path= f'{root_folder}/{volunteer_folder}/Distortion_Corrected/{divo_folder}'


        #original Image folder
        image_folder = os.path.join(divo_path, '03_Segmentation_Images')
        print(f'image_folder: {image_folder}')
        # Loop over slices

        avg_image_file = os.path.join(pwd,image_folder, f'Average_Diffusion_Weighted_Image_Slice_{slice}.nii')
        mean_diff_file = os.path.join(pwd,image_folder, f'Mean_Diffusivty_Image_Slice_{slice}.nii')
        eigenvector_file = os.path.join(pwd,image_folder, f'Primary_Eigenvector_Image_Slice_{slice}.nii')
        FA_file = os.path.join(pwd,image_folder, f'Fractional_Anisotropy_Image_Slice_{slice}.nii')
        

        # Check if all required files exist
        print(f'avg_image_file: {avg_image_file}')
        if all(os.path.exists(f) for f in [avg_image_file, mean_diff_file, eigenvector_file, FA_file]):
            # Load images and mask
            avg_image = nib.load(avg_image_file).get_fdata()
            mean_diff_image = nib.load(mean_diff_file).get_fdata()
            eigenvector_image = nib.load(eigenvector_file).get_fdata()
            FA_image = nib.load(FA_file).get_fdata()
            

            inference_mask = nib.load(mask_file_path).get_fdata()
            
            # Find bounding box from the mask
            coords = np.argwhere(inference_mask > 0)
            x_min, y_min = coords.min(axis=0)
            x_max, y_max = coords.max(axis=0) + 1  # Add 1 for inclusive cropping
            # Crop 



            # Apply mask cropping and resize to 256x256
            avg_cropped = avg_image[x_min:x_max, y_min:y_max]
            mean_diff_cropped = mean_diff_image[x_min:x_max, y_min:y_max]
            eigenvector_cropped = eigenvector_image[x_min:x_max, y_min:y_max, 0] + eigenvector_image[x_min:x_max, y_min:y_max, 1]
            FA_cropped = FA_image[x_min:x_max, y_min:y_max]



            avg_resized = cv2.resize(avg_cropped, (interpolation_size, interpolation_size), interpolation=cv2.INTER_LINEAR)
            mean_diff_resized = cv2.resize(mean_diff_cropped, (interpolation_size, interpolation_size), interpolation=cv2.INTER_LINEAR)
            eigenvector_resized = cv2.resize(eigenvector_cropped, (interpolation_size, interpolation_size), interpolation=cv2.INTER_LINEAR)
            FA_resized = cv2.resize(FA_cropped, (interpolation_size, interpolation_size), interpolation=cv2.INTER_LINEAR)
            
            
            
            # Save images with nnUNet naming scheme
            common_name_id = f'{root_folder}_{volunteer_folder}_{divo_folder}_slice_{slice}'
            nib.save(nib.Nifti1Image(avg_resized, np.eye(4)), 
                        os.path.join(output_image_folder, f'{common_name_id}_0000.nii.gz'))
            nib.save(nib.Nifti1Image(mean_diff_resized, np.eye(4)), 
                        os.path.join(output_image_folder, f'{common_name_id}_0001.nii.gz'))
            nib.save(nib.Nifti1Image(eigenvector_resized, np.eye(4)), 
                        os.path.join(output_image_folder, f'{common_name_id}_0002.nii.gz'))
            nib.save(nib.Nifti1Image(FA_resized, np.eye(4)), 
                        os.path.join(output_image_folder, f'{common_name_id}_0003.nii.gz'))

            print(f"Processed and saved {common_name_id}_0000 to _0003.nii.gz")
        else:
            print(f"Missing files for slice {slice} in {divo_folder}")



parts: ['Hannum', 'Volunteer', '01', 'DiVO', '06', '10', 'slice', '001']
Processing mask file: Hannum_Volunteer_01_DiVO_06_10_slice_001.nii.gz
Root folder: Hannum, Volunteer: Volunteer_01, DIVO folder: DiVO_06_10, Slice: 001
Path of mask: Hannum_Volunteer_01_DiVO_06_10_001
image_folder: Hannum/Volunteer_01/Distortion_Corrected/DiVO_06_10/03_Segmentation_Images
avg_image_file: /home/sastocke/nnUNet/inferenceTest/Hannum/Volunteer_01/Distortion_Corrected/DiVO_06_10/03_Segmentation_Images/Average_Diffusion_Weighted_Image_Slice_001.nii
Processed and saved Hannum_Volunteer_01_DiVO_06_10_slice_001_0000 to _0003.nii.gz


In [ ]:
#Segmentation 2

input_dir2 = "/home/sastocke/nnUNet/inferenceTest/processedOutPut1/imagesTr"
output_2 = "/home/sastocke/nnUNet/inferenceTest/OutputnnUnet2"
# Set the environment variables
os.environ["nnUNet_raw"] = "/home/sastocke/nnUNet/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/home/sastocke/nnUNet/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/home/sastocke/nnUNet/nnUNet_results"


os.system((f"nnUNetv2_predict -i {input_dir2} -o {output_2}  --save_probabilities -d 36 -c '2d' -f 5"))


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting Hannum_Volunteer_01_DiVO_06_10_slice_001:
perform_everything_on_device: True


100%|██████████| 1/1 [00:09<00:00,  9.26s/it]


sending off prediction to background worker for resampling and export
done with Hannum_Volunteer_01_DiVO_06_10_slice_001


0

In [ ]:
#Post processing up to original size again:


